# XGBoost CV

참고 : https://www.kaggle.com/aharless/xgboost-cv-lb-284

# 1st

In [1]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  # OPTIMIZE_ROUNDS가 설정된 경우 EARLY_STOPPING_ROuNDS를 높게 설정
# 빨리 정지하고 싶다면 EARLY_STOPPING_ROUNDS를 줄임

처음에 MAX_ROUNDS를 아주 높게 설정하고 OPTIMIZE_ROUNDS를 사용해 적절한 라운드 수를 파악할 것을 권장한다. (round는 모든 fold 중 best_ntree_limit의 최대값과 가까워야하고, 아마 모델이 적절하게 정규화되면 조금 더 높을 수도 있다. 혹은 대체로, verbose=True로 설정하고 세부사항을 확인해 모든 fold에 잘 작동하는 round의 수를 찾을 수 있다.) 그런 다음, OPTIMAIZE_ROUNDS를 끄고 MAX_ROUNDS를 총 round의 적절한 수로 설정한다.<br><br>
각 fold에 가장 적합한 round를 골라 "early stopping"하는 것의 문제는 검증용 데이터에 과적합한다는 것이다. 따라서 test 데이터를 예측하는 최적의 모델을 생성하지 않을 수 있고, 다른 모델과 stacking/ensembling할 검증 데이터를 생성하는 데 사용되면, 앙상블에 가중치가 너무 많이 실린다. 또 다른 가능성은 최적의 round보다 조기 멈춤이 실제로 일어나는 round를 사용하는 것이다. 이렇게 하면 과적합 문제는 해결되지만, 아직 도움이 되진 않는다. (모든 fold의 일정한 round 수보다 fold당 20 round의 조기 멈춤이 검증 점수를 더 악화하므로, 조기멈춤은 실제로 언더피팅되는 것 같다.)

In [8]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
#from numba import jit
import time
import gc

In [9]:
# gini 계산

#@jit
def eval_gini(y_true,y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(ya_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    
    for i in range(n-1,-1,-1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2*gini / (ntrue*(n-ntrue))
    
    return gini

In [30]:
def gini_xgb(pres,dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels,preds)
    return [('gini',gini_score)]

def add_noise(series, noise_level):
    return series * (1+noise_level*np.random.randn(len(series)))

def target_encode(trn_series=None,val_series=None,tst_series=None,target=None,min_samples_leaf=1,smoothing=1,noise_level=0):
    # trn_series : train의 범주형 변수
    # tst_series : test의 범주형 변수
    # min_samples_leaf(int) : 범주 평균을 취하는 최소 샘플 수
    # smoothing(int) : 범주 평균과 이전 값의 균형을 맞추는 smoothing 효과
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series,target],axis=1)
    
    # target 평균 계산
    averages = temp.groupby(by=trn_series.name)[target.name].agg(['mean','count'])
    # smoothing 계산
    smoothing = 1 / (1+np.exp(-(averages['count']-min_samples_leaf) / smoothing))
    # 모든 target 데이터에 평균 함수 적용
    prior = target.mean()
    # count가 클수록 full_avg가 적게 고려됨
    averages[target.name] = prior * (1-smoothing) + averages['mean'] * smoothing
    averages.drop(['mean','count'],axis=1,inplace=True)
    
    # train series에 평균 적용
    ft_trn_series = pd.merge(trn_series.to_frame(trn_series.name),
                            averages.reset_index().rename(columns={'index':target.name,target.name:'average'}),
                            on = trn_series.name, how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_trn_series.index = trn_series.index
    # validation series에 평균 적용
    ft_val_series = pd.merge(val_series.to_frame(val_series.name),
                            averages.reset_index().rename(columns={'index':target.name,target.name:'average'}),
                            on = val_series.name, how='left')['average'].rename(val_series.name + '_mean').fillna(prior)
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_val_series.index = val_series.index
    # test series에 평균 적용
    ft_tst_series = pd.merge(val_series.to_frame(val_series.name),
                             averages.reset_index().rename(columns={'index':target.name,target.name:'average'}),
                             on=val_series.name, how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_tst_series.index = val_series.index
    
    return add_noise(ft_trn_series, noise_level),add_noise(ft_val_series, noise_level),add_noise(ft_tst_series, noise_level)

In [15]:
# 데이터 불러오기
train_df = pd.read_csv('../input/train.csv',na_values='-1')  # .iloc[0:200,:]
test_df = pd.read_csv('../input/test.csv',na_values='-1')

In [13]:
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
    "ps_reg_03",  #            : 1408.42 / shadow  511.15
    "ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
    "ps_ind_03",  #            : 1219.47 / shadow  230.55
    "ps_ind_15",  #            :  922.18 / shadow  242.00
    "ps_reg_02",  #            :  920.65 / shadow  267.50
    "ps_car_14",  #            :  798.48 / shadow  549.58
    "ps_car_12",  #            :  731.93 / shadow  293.62
    "ps_car_01_cat",  #        :  698.07 / shadow  178.72
    "ps_car_07_cat",  #        :  694.53 / shadow   36.35
    "ps_ind_17_bin",  #        :  620.77 / shadow   23.15
    "ps_car_03_cat",  #        :  611.73 / shadow   50.67
    "ps_reg_01",  #            :  598.60 / shadow  178.57
    "ps_car_15",  #            :  593.35 / shadow  226.43
    "ps_ind_01",  #            :  547.32 / shadow  154.58
    "ps_ind_16_bin",  #        :  475.37 / shadow   34.17
    "ps_ind_07_bin",  #        :  435.28 / shadow   28.92
    "ps_car_06_cat",  #        :  398.02 / shadow  212.43
    "ps_car_04_cat",  #        :  376.87 / shadow   76.98
    "ps_ind_06_bin",  #        :  370.97 / shadow   36.13
    "ps_car_09_cat",  #        :  214.12 / shadow   81.38
    "ps_car_02_cat",  #        :  203.03 / shadow   26.67
    "ps_ind_02_cat",  #        :  189.47 / shadow   65.68
    "ps_car_11",  #            :  173.28 / shadow   76.45
    "ps_car_05_cat",  #        :  172.75 / shadow   62.92
    "ps_calc_09",  #           :  169.13 / shadow  129.72
    "ps_calc_05",  #           :  148.83 / shadow  120.68
    "ps_ind_08_bin",  #        :  140.73 / shadow   27.63
    "ps_car_08_cat",  #        :  120.87 / shadow   28.82
    "ps_ind_09_bin",  #        :  113.92 / shadow   27.05
    "ps_ind_04_cat",  #        :  107.27 / shadow   37.43
    "ps_ind_18_bin",  #        :   77.42 / shadow   25.97
    "ps_ind_12_bin",  #        :   39.67 / shadow   15.52
    "ps_ind_14",  #            :   37.37 / shadow   16.65
]

# combinations 추가
combs = [('ps_reg_01','ps_car_02_cat'), ('ps_reg_01','ps_car_04_cat')]

In [17]:
# 데이터 처리

id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1,f2) in enumerate(combs):
    name1 = f1 + '_plus' + f2
    print('current featrues %60s %4d in %5.1f' % (name1,n_c+1,(time.time()-start)/60),end='')
    print('\r'*75, end='')
    
    ### train,data의 name1 변수 추가 : combs의 두 변수값을 문자화해 합침
    train_df[name1] = train_df[f1].apply(lambda x:str(x)) + '_' + train_df[f2].apply(lambda x:str(x))
    test_df[name1] = test_df[f1].apply(lambda x:str(x)) + '_' + test_df[f2].apply(lambda x: str(x))
    
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))
    
    train_features.append(name1)
    
    X = train_df[train_features]
    test_df = test_df[train_features]
    
    f_cats = [f for f in X.columns if '_cat' in f]

current featrues                                  ps_reg_01_plusps_car_04_cat    2 in   0.1

In [18]:
y_valid_pred = 0*y
y_test_pred = 0

In [19]:
# fold 설정
K = 5
kf = KFold(n_splits=K, random_state=1, shuffle=True)
np.random.seed(0)

In [20]:
# 분류기 설정
model = XGBClassifier(n_estimators=MAX_ROUNDS, max_depth=4, objective='binary:logistic',
                     learning_rete=LEARNING_RATE, subsample=.8, min_child_weight=6,
                     colsample_bytree=.8, scale_pos_weight=1.6, gamma=10, reg_alpha=8, reg_lambda=1.3)

In [ ]:
import warnings
warnings.filterwarnings('ignore')


# CV 작동
for i,(train_index,test_index) in enumerate(kf.split(train_df)):
    # 이 fold로 데이터 생성
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print('\nFold ',i)
    
    # 데이터 encode
    for f in f_cats:
        X_train[f+'_avg'], X_valid[f+'_avg'], X_test[f+'_avg'] = target_encode(
        trn_series=X_train[f], val_series=X_valid[f], tst_series=X_test[f],
        target=y_train, min_samples_leaf=50, smoothing=5, noise_level=0)
        
    # 이 fold로 모델 작동
    if OPTIMIZE_ROUNDS:
        eval_set = [(X_valid,y_valid)]
        fit_model = model.fit(X_train,y_train,eval_set=eval_set,eval_metric=gini_xgb,
                             early_stopping_rounds=EARLY_STOPPING_ROUNDS,verbose=False)
        print(' Best N trees = ', model.best_ntree_limit)
        print(' Best gini = ', model.best_score)
    else:
        fit_model = model.fit(X_train,y_train)
        
    # 이 fold로 validation 데이터 예측 생성
    pred = fit_model.predict_proba(X_valid)[:,1]
    print(' Gini = ', eval_gini(y_valid,pred))
    y_valid_pred.iloc[test_index] = pred
    
    # test set 예측값 축적
    y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
    del X_test,X_train,X_valid,y_train
    
y_test_pred /= K  # test set 예측값 평균

print('\nGini for full training set:')
eval_gini(y, y_valid_pred)

위의 셀 너무 오래 걸려서 중지

In [33]:
# staking/ensembling을 위해 validation 예측 저장
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('xgb_valid.csv',float_format='%.6f',index=False)

In [34]:
# 제출 파일 생성
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('xgb_submit.csv', float_format='%.6f', index=False)

# 2nd

In [35]:
MAX_ROUNDS = 400
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  # OPTIMIZE_ROUNDS가 설정된 경우 EARLY_STOPPING_ROuNDS를 높게 설정
# 빨리 정지하고 싶다면 EARLY_STOPPING_ROUNDS를 줄임

처음에 MAX_ROUNDS를 아주 높게 설정하고 OPTIMIZE_ROUNDS를 사용해 적절한 라운드 수를 파악할 것을 권장한다. (round는 모든 fold 중 best_ntree_limit의 최대값과 가까워야하고, 아마 모델이 적절하게 정규화되면 조금 더 높을 수도 있다. 혹은 대체로, verbose=True로 설정하고 세부사항을 확인해 모든 fold에 잘 작동하는 round의 수를 찾을 수 있다.) 그런 다음, OPTIMAIZE_ROUNDS를 끄고 MAX_ROUNDS를 총 round의 적절한 수로 설정한다.<br><br>
각 fold에 가장 적합한 round를 골라 "early stopping"하는 것의 문제는 검증용 데이터에 과적합한다는 것이다. 따라서 test 데이터를 예측하는 최적의 모델을 생성하지 않을 수 있고, 다른 모델과 stacking/ensembling할 검증 데이터를 생성하는 데 사용되면, 앙상블에 가중치가 너무 많이 실린다. 또 다른 가능성은 최적의 round보다 조기 멈춤이 실제로 일어나는 round를 사용하는 것이다. 이렇게 하면 과적합 문제는 해결되지만, 아직 도움이 되진 않는다. (모든 fold의 일정한 round 수보다 fold당 20 round의 조기 멈춤이 검증 점수를 더 악화하므로, 조기멈춤은 실제로 언더피팅되는 것 같다.)

In [36]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
#from numba import jit
import time
import gc

In [37]:
# gini 계산

#@jit
def eval_gini(y_true,y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(ya_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    
    for i in range(n-1,-1,-1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2*gini / (ntrue*(n-ntrue))
    
    return gini

In [38]:
def gini_xgb(pres,dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels,preds)
    return [('gini',gini_score)]

def add_noise(series, noise_level):
    return series * (1+noise_level*np.random.randn(len(series)))

def target_encode(trn_series=None,val_series=None,tst_series=None,target=None,min_samples_leaf=1,smoothing=1,noise_level=0):
    # trn_series : train의 범주형 변수
    # tst_series : test의 범주형 변수
    # min_samples_leaf(int) : 범주 평균을 취하는 최소 샘플 수
    # smoothing(int) : 범주 평균과 이전 값의 균형을 맞추는 smoothing 효과
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series,target],axis=1)
    
    # target 평균 계산
    averages = temp.groupby(by=trn_series.name)[target.name].agg(['mean','count'])
    # smoothing 계산
    smoothing = 1 / (1+np.exp(-(averages['count']-min_samples_leaf) / smoothing))
    # 모든 target 데이터에 평균 함수 적용
    prior = target.mean()
    # count가 클수록 full_avg가 적게 고려됨
    averages[target.name] = prior * (1-smoothing) + averages['mean'] * smoothing
    averages.drop(['mean','count'],axis=1,inplace=True)
    
    # train series에 평균 적용
    ft_trn_series = pd.merge(trn_series.to_frame(trn_series.name),
                            averages.reset_index().rename(columns={'index':target.name,target.name:'average'}),
                            on = trn_series.name, how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_trn_series.index = trn_series.index
    # validation series에 평균 적용
    ft_val_series = pd.merge(val_series.to_frame(val_series.name),
                            averages.reset_index().rename(columns={'index':target.name,target.name:'average'}),
                            on = val_series.name, how='left')['average'].rename(val_series.name + '_mean').fillna(prior)
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_val_series.index = val_series.index
    # test series에 평균 적용
    ft_tst_series = pd.merge(val_series.to_frame(val_series.name),
                             averages.reset_index().rename(columns={'index':target.name,target.name:'average'}),
                             on=val_series.name, how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge는 인덱스를 유지하지 않으므로 복원
    ft_tst_series.index = val_series.index
    
    return add_noise(ft_trn_series, noise_level),add_noise(ft_val_series, noise_level),add_noise(ft_tst_series, noise_level)

In [39]:
# 데이터 불러오기
train_df = pd.read_csv('../input/train.csv',na_values='-1')  # .iloc[0:200,:]
test_df = pd.read_csv('../input/test.csv',na_values='-1')

In [40]:
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
    "ps_reg_03",  #            : 1408.42 / shadow  511.15
    "ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
    "ps_ind_03",  #            : 1219.47 / shadow  230.55
    "ps_ind_15",  #            :  922.18 / shadow  242.00
    "ps_reg_02",  #            :  920.65 / shadow  267.50
    "ps_car_14",  #            :  798.48 / shadow  549.58
    "ps_car_12",  #            :  731.93 / shadow  293.62
    "ps_car_01_cat",  #        :  698.07 / shadow  178.72
    "ps_car_07_cat",  #        :  694.53 / shadow   36.35
    "ps_ind_17_bin",  #        :  620.77 / shadow   23.15
    "ps_car_03_cat",  #        :  611.73 / shadow   50.67
    "ps_reg_01",  #            :  598.60 / shadow  178.57
    "ps_car_15",  #            :  593.35 / shadow  226.43
    "ps_ind_01",  #            :  547.32 / shadow  154.58
    "ps_ind_16_bin",  #        :  475.37 / shadow   34.17
    "ps_ind_07_bin",  #        :  435.28 / shadow   28.92
    "ps_car_06_cat",  #        :  398.02 / shadow  212.43
    "ps_car_04_cat",  #        :  376.87 / shadow   76.98
    "ps_ind_06_bin",  #        :  370.97 / shadow   36.13
    "ps_car_09_cat",  #        :  214.12 / shadow   81.38
    "ps_car_02_cat",  #        :  203.03 / shadow   26.67
    "ps_ind_02_cat",  #        :  189.47 / shadow   65.68
    "ps_car_11",  #            :  173.28 / shadow   76.45
    "ps_car_05_cat",  #        :  172.75 / shadow   62.92
    "ps_calc_09",  #           :  169.13 / shadow  129.72
    "ps_calc_05",  #           :  148.83 / shadow  120.68
    "ps_ind_08_bin",  #        :  140.73 / shadow   27.63
    "ps_car_08_cat",  #        :  120.87 / shadow   28.82
    "ps_ind_09_bin",  #        :  113.92 / shadow   27.05
    "ps_ind_04_cat",  #        :  107.27 / shadow   37.43
    "ps_ind_18_bin",  #        :   77.42 / shadow   25.97
    "ps_ind_12_bin",  #        :   39.67 / shadow   15.52
    "ps_ind_14",  #            :   37.37 / shadow   16.65
]

# combinations 추가
combs = [('ps_reg_01','ps_car_02_cat'), ('ps_reg_01','ps_car_04_cat')]

In [41]:
# 데이터 처리

id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1,f2) in enumerate(combs):
    name1 = f1 + '_plus' + f2
    print('current featrues %60s %4d in %5.1f' % (name1,n_c+1,(time.time()-start)/60),end='')
    print('\r'*75, end='')
    
    ### train,data의 name1 변수 추가 : combs의 두 변수값을 문자화해 합침
    train_df[name1] = train_df[f1].apply(lambda x:str(x)) + '_' + train_df[f2].apply(lambda x:str(x))
    test_df[name1] = test_df[f1].apply(lambda x:str(x)) + '_' + test_df[f2].apply(lambda x: str(x))
    
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))
    
    train_features.append(name1)
    
    X = train_df[train_features]
    test_df = test_df[train_features]
    
    f_cats = [f for f in X.columns if '_cat' in f]

current featrues                                  ps_reg_01_plusps_car_04_cat    2 in   0.1

In [42]:
y_valid_pred = 0*y
y_test_pred = 0

In [43]:
# fold 설정
K = 5
kf = KFold(n_splits=K, random_state=1, shuffle=True)
np.random.seed(0)

In [44]:
# 분류기 설정
model = XGBClassifier(n_estimators=MAX_ROUNDS, max_depth=4, objective='binary:logistic',
                     learning_rete=LEARNING_RATE, subsample=.8, min_child_weight=6,
                     colsample_bytree=.8, scale_pos_weight=1.6, gamma=10, reg_alpha=8, reg_lambda=1.3)

In [ ]:
# CV 작동
for i,(train_index,test_index) in enumerate(kf.split(train_df)):
    # 이 fold로 데이터 생성
    y_train, y_valid = y.iloc[train_index].copy(), y.iloc[test_index]
    X_train, X_valid = X.iloc[train_index,:].copy(), X.iloc[test_index,:].copy()
    X_test = test_df.copy()
    print('\nFold ',i)
    
    # 데이터 encode
    for f in f_cats:
        X_train[f+'_avg'], X_valid[f+'_avg'], X_test[f+'_avg'] = target_encode(
        trn_series=X_train[f], val_series=X_valid[f], tst_series=X_test[f],
        target=y_train, min_samples_leaf=200, smoothing=10, noise_level=0)
        
    # 이 fold로 모델 작동
    if OPTIMIZE_ROUNDS:
        eval_set = [(X_valid,y_valid)]
        fit_model = model.fit(X_train,y_train,eval_set=eval_set,eval_metric=gini_xgb,
                             early_stopping_rounds=EARLY_STOPPING_ROUNDS,verbose=False)
        print(' Best N trees = ', model.best_ntree_limit)
        print(' Best gini = ', model.best_score)
    else:
        fit_model = model.fit(X_train,y_train)
        
    # 이 fold로 validation 데이터 예측 생성
    pred = fit_model.predict_proba(X_valid)[:,1]
    print(' Gini = ', eval_gini(y_valid,pred))
    y_valid_pred.iloc[test_index] = pred
    
    # test set 예측값 축적
    y_test_pred += fit_model.predict_proba(X_test)[:,1]
    
    del X_test,X_train,X_valid,y_train
    
y_test_pred /= K  # test set 예측값 평균

print('\nGini for full training set:')
eval_gini(y, y_valid_pred)

위 셀 너무 오래 걸려서 중지

In [45]:
# staking/ensembling을 위해 validation 예측 저장
val = pd.DataFrame()
val['id'] = id_train
val['target'] = y_valid_pred.values
val.to_csv('xgb_valid.csv',float_format='%.6f',index=False)

In [46]:
# 제출 파일 생성
sub = pd.DataFrame()
sub['id'] = id_test
sub['target'] = y_test_pred
sub.to_csv('xgb_submit.csv', float_format='%.6f', index=False)